# Análise de Vendas - E-commerce (Amazon) EXECUTE ESSE PROJETO NO GOOGLE COLAB

Este notebook (Google Colab) conduz uma análise de dados de vendas de e-commerce com base em um conjunto de dados da Amazon, utilizando técnicas de visualização, tratamento de dados e modelagem preditiva.

## Etapas do Notebook

1. **Instalação de pacotes**: Instalação dos pacotes `kagglehub` e `openpyxl` necessários para baixar e ler os dados.
2. **Download do dataset**: Utilização da API do KaggleHub para baixar o dataset de vendas.
3. **Importações e exploração inicial de arquivos**: Importação de bibliotecas padrão e listagem dos arquivos disponíveis.
4. **Leitura dos dados**: Leitura do arquivo CSV com as vendas da Amazon.
5. **Dimensão dos dados**: Verifica número de linhas e colunas.
6. **Informações do dataframe**: Estrutura e tipos de dados.
7. **Tipos das colunas**: Lista os tipos de dados das colunas.
8. **Estatísticas numéricas**: Resumo estatístico de colunas numéricas.
9. **Estatísticas categóricas**: Resumo de colunas categóricas.
10. **Valores ausentes**: Quantificação de dados faltantes.
11. **Conversão da data**: Transforma a coluna 'Date' para o tipo datetime.
12. **Extração de componentes da data**: Extrai ano, mês, dia e dia da semana da data.
13. **Teste ANOVA**: Verifica se há diferença significativa nas vendas entre categorias.
14. **Criação de variáveis preditoras**: Define as variáveis para o modelo e aplica codificação one-hot.
15. **Divisão treino/teste**: Separação dos dados para treino e teste.
16. **Tratamento de dados ausentes**: Limpeza e preenchimento de dados faltantes nas colunas principais.
17. **Visualização 1 - Tendência de vendas por mês**.
18. **Visualização 2 - Distribuição do valor de vendas (Amount)**.
19. **Visualização 3 - Boxplot de vendas por categoria**.
20. **Visualização 4 - Correlação entre variáveis numéricas**.
21. **Modelo de Regressão Linear**: Treinamento, previsão e avaliação com métricas de erro.
22. **Modelo Random Forest**: Treinamento e avaliação com Random Forest.
23. **Random Forest com transformação logarítmica**: Aplica log nos dados para melhorar performance.
24. **Random Forest com remoção de outliers**: Remove outliers no target e reavalia o modelo.
25. **Gradient Boosting**: Modelo de regressão com Gradient Boosting e avaliação.
26. **Comparação de modelos**: Compila métricas MAE, RMSE e R² de todos os modelos.
27. **Importância de variáveis**: Mostra as 10 variáveis mais relevantes no modelo Random Forest.

In [ ]:
!pip install kagglehub --quiet
!pip install openpyxl --quiet

In [ ]:
import kagglehub

# Baixa o dataset
path = kagglehub.dataset_download("thedevastator/unlock-profits-with-e-commerce-sales-data")
print("Caminho:", path)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")

# Ver arquivos disponíveis
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv(os.path.join(path, "Amazon Sale Report.csv"))
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

In [ ]:
df.isnull().sum()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%m-%d-%y', errors='coerce')

In [ ]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['weekday'] = df['Date'].dt.weekday
df[['Date','year','month','day','weekday']].head()

In [ ]:
from scipy.stats import f_oneway

groups = [group['Amount'].values for _, group in df.groupby('Category')]
f_stat, p_value = f_oneway(*groups)
print(f"ANOVA F={f_stat:.2f}, p-valor={p_value:.5f}")
if p_value < 0.05:
    print("→ Diferenças significativas nas vendas entre categorias.")
else:
    print("→ Nenhuma diferença significativa entre as médias.")

In [ ]:
features = ['month', 'weekday', 'Qty', 'Category', 'Fulfilment', 'ship-service-level']
X = df[features]
y = df['Amount']
X = pd.get_dummies(X, columns=['Category', 'Fulfilment', 'ship-service-level'], drop_first=True)
print(f"Shape final das features: {X.shape}")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")

In [ ]:
print(df.isnull().sum())
df = df.dropna(subset=['Amount', 'currency'])
df['ship-city'] = df['ship-city'].fillna('Desconhecido')
df['ship-state'] = df['ship-state'].fillna('Desconhecido')
df['ship-postal-code'] = df['ship-postal-code'].fillna(0).astype(int)
df['ship-country'] = df['ship-country'].fillna('Desconhecido')
print("\nApós tratamento:")
print(df.isnull().sum())

In [ ]:
# Visualizações
import matplotlib.pyplot as plt
import seaborn as sns

vendas_por_mes = df.groupby('month')['Amount'].sum()
plt.figure(figsize=(8,4))
plt.plot(vendas_por_mes.index, vendas_por_mes.values)
plt.title('Total de Vendas por Mês')
plt.xlabel('Mês')
plt.ylabel('Valor Total (INR)')
plt.xticks(range(1,13))
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['Amount'], bins=50, kde=True)
plt.title('Distribuição do Valor de Venda')
plt.xlabel('Amount')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='Category', y='Amount', data=df)
plt.title('Boxplot de Vendas por Categoria')
plt.xticks(rotation=45)
plt.show()

In [ ]:
corr = df[['year','month','day','weekday','Qty','Amount']].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, fmt='.2f')
plt.title('Mapa de Correlação')
plt.show()

In [ ]:
# Modelagem: Regressão Linear
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X = pd.get_dummies(df[features], columns=['Category', 'Fulfilment', 'ship-service-level'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE:  {mae:.2f}\nRMSE: {rmse:.2f}\nR²:   {r2:.2f}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mae_rf  = mean_absolute_error(y_test, y_pred_rf)
mse_rf  = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf   = r2_score(y_test, y_pred_rf)
print(f"RF MAE:  {mae_rf:.2f}\nRF RMSE: {rmse_rf:.2f}\nRF R²:   {r2_rf:.2f}")

In [ ]:
y_train_log = np.log1p(y_train)
y_test_log  = np.log1p(y_test)
rf_log = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_log.fit(X_train, y_train_log)
y_pred_log = rf_log.predict(X_test)
y_pred_orig = np.expm1(y_pred_log)
mae_log = mean_absolute_error(y_test, y_pred_orig)
rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_orig))
r2_log = r2_score(y_test, y_pred_orig)
print(f"RF+Log MAE:  {mae_log:.2f}\nRF+Log RMSE: {rmse_log:.2f}\nRF+Log R²:   {r2_log:.2f}")

In [ ]:
limite = df['Amount'].quantile(0.99)
df_filtrado = df[df['Amount'] <= limite]
Xf = pd.get_dummies(df_filtrado[features], columns=['Category','Fulfilment','ship-service-level'], drop_first=True)
yf = df_filtrado['Amount']
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(Xf, yf, test_size=0.2, random_state=42)
rf_o = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_o.fit(X_train_f, y_train_f)
yp_o = rf_o.predict(X_test_f)
mae_o = mean_absolute_error(y_test_f, yp_o)
rmse_o = np.sqrt(mean_squared_error(y_test_f, yp_o))
r2_o = r2_score(y_test_f, yp_o)
print(f"RF sem outliers MAE:  {mae_o:.2f}\nRF sem outliers RMSE: {rmse_o:.2f}\nRF sem outliers R²:   {r2_o:.2f}")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
gb.fit(X_train_gb, y_train_gb)
y_pred_gb = gb.predict(X_test_gb)
mae_gb  = mean_absolute_error(y_test_gb, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_test_gb, y_pred_gb))
r2_gb   = r2_score(y_test_gb, y_pred_gb)
print(f"GB MAE:  {mae_gb:.2f}\nGB RMSE: {rmse_gb:.2f}\nGB R²:   {r2_gb:.2f}")

In [ ]:
resultados = pd.DataFrame({
    'Modelo': ['Linear', 'RF', 'RF sem outliers', 'GB'],
    'MAE':    [mae,     mae_rf,      mae_o,            mae_gb],
    'RMSE':   [rmse,    rmse_rf,     rmse_o,           rmse_gb],
    'R2':     [r2,      r2_rf,       r2_o,             r2_gb]
})
display(resultados.style.format({'MAE': '{:.2f}', 'RMSE': '{:.2f}', 'R2': '{:.2f}'}))

In [ ]:
importancias = rf_o.feature_importances_
nomes_feats = X_train_f.columns
feat_imp = pd.Series(importancias, index=nomes_feats).sort_values(ascending=False).head(10)
plt.figure(figsize=(8,6))
feat_imp.plot(kind='bar')
plt.title('Top 10 Features mais relevantes (RF sem outliers)')
plt.ylabel('Importância')
plt.tight_layout()
plt.show()